# Relating band members to the individual artists

Demonstrate doing this using a graph, and see how fast it is

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import requests
import pprint
from pprint import pprint as pp

## Get some data

We get a list of names which we'll pretend are "artists".
Names are easy for humans to work with/read/remember.

In [2]:
r = requests.get('https://raw.githubusercontent.com/dominictarr/random-name/master/first-names.txt')
r.raise_for_status()
names = r.text.strip().splitlines() 

In [3]:
print(len(names))
pp(names[:5])

4945
['Aaren', 'Aarika', 'Abagael', 'Abagail', 'Abbe']


In [4]:
def make_bands(names, n_bands, max_bandsize):
    """Randomly combine names into lists of names containin between 2 and max_bandsize names"""
    band_size_arr = np.random.randint(2, max_bandsize, (n_bands,))
    bands = list()
    for bs in band_size_arr:
        bands.append(np.random.choice(names, bs).tolist())
    return bands


# Check that it works
make_bands(names, 5, 8)

[['Willette', 'Vi'],
 ['Jena', 'Noelani', 'Verena'],
 ['Gabriellia', 'Constancy', 'Tuesday', 'Jorrie', 'Edie', 'Marika', 'Holly'],
 ['Charlena', 'Aurora', 'Tiffy'],
 ['Monica', 'Angelita', 'Adan', 'Marketa', 'Bidget', 'Sidoney']]

In [5]:
df = pd.Series(make_bands(names, 10_000, 9), name="Bands").to_frame() 

In [6]:
df.head() 

,Bands
0,"[Rivy, Nerty]"
1,"[Tessi, Nanete, Daniella, Almire, Lucilia]"
2,"[Arabelle, Alethea]"
3,"[Charmine, Vin, Aimil, Aimil, Phyllis, Mira, C..."
4,"[Maisey, Chandra, Junie]"


This is the base data foundation.
We can now chose to ignore the ordering of the names in the band names, or preserve them — by casting the `Bands`-columns to use frozenset, we ignore the ordering, but if we were to use tuple instead, we would perserve them.
`forzenset` is an immutable (and thus hashable) version if a `set`, just like `tuple` is an immutable and hashable version of a `list`.

In [7]:
df['Bands'] = df.Bands.apply(frozenset)  # tuple is an alternative for preserving order of band members

In [8]:
df.head() 

,Bands
0,"(Nerty, Rivy)"
1,"(Almire, Daniella, Lucilia, Tessi, Nanete)"
2,"(Arabelle, Alethea)"
3,"(Vin, Cathy, Coretta, Phyllis, Aimil, Mira, Ch..."
4,"(Maisey, Junie, Chandra)"


## Construct the bands and bandmembers graph

In [9]:
%%time

g = nx.Graph()
for band in df.Bands:
    for artist in band:
        g.add_edge(band, artist)

CPU times: user 166 ms, sys: 10.8 ms, total: 176 ms
Wall time: 176 ms


Now test it

In [10]:
# Number of bands Catriona are participating in
g.degree('Catriona')

9

In [11]:
# Let's get a list of her bands (the g.neighbours returns an iterator, thus the call to list)

print(*list(g.neighbors('Catriona')), sep='\n')

frozenset({'Alyson', 'Christy', 'Thomasin', 'Fiann', 'Kitti', 'Caril', 'Catriona'})
frozenset({'Naoma', 'Lucy', 'Jo-Ann', 'Danny', 'Clea', 'Catriona', 'Cammie', 'Laetitia'})
frozenset({'Johanna', 'Adele', 'Catriona', 'Paloma', 'Rosalie'})
frozenset({'Eleonora', 'Livvy', 'Drusie', 'Amelia', 'Clovis', 'Catriona', 'Amandi'})
frozenset({'Catriona', 'Marlene', 'Claribel', 'Eydie'})
frozenset({'Angil', 'Margarethe', 'Micki', 'Zondra', 'Chelsey', 'Catriona'})
frozenset({'Gavrielle', 'Carlita', 'Marianna', 'Gilbertine', 'Brenda', 'Catriona', 'Zorine', 'Jackqueline'})
frozenset({'Hedvige', 'Philipa', 'Angele', 'Angelique', 'Kathrine', 'Dyanna', 'Catriona', 'Kylila'})
frozenset({'Catriona', 'Austin'})


In [15]:
de = nx.to_pandas_edgelist(g)
de.head() 

,source,target
0,"(Nerty, Rivy)",Nerty
1,"(Nerty, Rivy)",Rivy
2,Nerty,"(Nerty, Mady, Pierette)"
3,Nerty,"(Nerty, Clio, Jillene, Yvette)"
4,Nerty,"(Oneida, Nerty, Lynette, Rhody, Carey)"


In [30]:
nodes_str = (n for n in g.nodes if isinstance(n, str))
dct = {n: pd.Series(g.neighbors(n)) for n in nodes_str}   
df_artists = pd.DataFrame(dct).T
df_artists.head() 

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Nerty,"(Nerty, Rivy)","(Nerty, Mady, Pierette)","(Nerty, Clio, Jillene, Yvette)","(Oneida, Nerty, Lynette, Rhody, Carey)","(Gale, Nerty, Beckie, Frederique, Amanda, Chri...","(Adele, Dyane, Abra, Morgana, Nerty, Fiorenze,...","(Marietta, Marie-Ann, Mitzi, Gwynne, Nerty, Zo...","(Una, Nerty, Milicent, Mahalia, Aimee, Winni, ...","(Blondelle, Marcile, Nerty, Tiffy, Janeen, Gol...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rivy,"(Nerty, Rivy)","(Estell, Madelene, Waneta, Laraine, Rivy, Adey...","(Hyacintha, Madlen, Noni, Aeriel, Salaidh, Riv...","(Candida, Zorana, Gabey, Rivy, Adel, Cynthia)","(Emmeline, Mead, Minnnie, Rivy)","(Nessie, Jobye, Rivy)","(Chantal, Jewell, Phylys, Rivy, Donella, Catie)","(Zaria, Winni, Rivy)","(Justinn, Rivy)","(Oralee, Gena, Amalle, Twila, Norean, Rivy, Ra...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Almire,"(Almire, Daniella, Lucilia, Tessi, Nanete)","(Almire, Stevana, Elfrida, Brunhilde, Moyra, C...","(Almire, Theresa, Nedda)","(Almire, Delia, Aviva, Lacy, Shari)","(Almire, Desiree, Ilka, Britt, Pansy, Evonne, ...","(Veda, Almire, Heidi, Kai, Joella, Valery, Agn...","(Almire, Anastassia, Cornela, Joelynn)","(Almire, Kaile, Pooh, Holly, Kati)","(Almire, Laurel, Querida, Codi, Terrie, Emera,...","(Almire, Emylee, Marilee, Sheri, Averyl, Zahar...",...,"(Almire, Theadora)","(Almire, Dinnie, Guenevere, Jilly, Opal, Janne...","(Almire, Debra, Loren, Dori, Sissy, Hope, Fide...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
Daniella,"(Almire, Daniella, Lucilia, Tessi, Nanete)","(Daniella, Netta, Laurene, Suzanna, Domeniga, ...","(Daniella, Yolanthe, Berti, Flossie, Phylis, R...","(Daniella, Zondra, Josepha, Delphine, Willette)","(Daniella, Kissie, Luce, Kacie, Rosette, Ladon...","(Daniella, Angela, Erminia, Stacia, Jacintha)","(Daniella, Lynette, Gratiana, Valma, Josey, Do...","(Billie, Daniella, Ophelie, Halley, Codee, Cor...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lucilia,"(Almire, Daniella, Lucilia, Tessi, Nanete)","(Ida, Jere, Lucilia, Amabel)","(Lucilia, Lanie, Aura, Kelsey, Shannen, Marena)","(Lucilia, Una, Lyda, TEirtza, Jaimie, Ramona)","(Whitney, Marj, Lucilia, Francoise, Crystie, V...","(Augustina, Maxy, Eloise, Lucilia, Danielle, R...","(Lucilia, Netta, Charisse, Erena, Erina, Steph...","(Lucilia, Lucy, Rosy, Dee, Cher, Myrilla)",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
